In [2]:
from my_import import *
df_train = pd.read_csv('df_train.csv')
df_val = pd.read_csv('df_val.csv')
df_test = pd.read_csv('df_test.csv')

#Make sure the genre collumns is in lists not strings
#NEED TO DO THIS EVERYTIME EXPORT DATASET
df_train['genres'] = df_train['genres'].apply(lambda x: list(ast.literal_eval(x)))
df_val['genres'] = df_val['genres'].apply(lambda x: list(ast.literal_eval(x)))
df_test['genres'] = df_test['genres'].apply(lambda x: list(ast.literal_eval(x)))

c:\Users\User\anaconda3\envs\NLP_CW\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(tokens)

df_train['clean_synopsis'] = df_train['synopsis'].apply(clean_text)
df_val['clean_synopsis'] = df_val['synopsis'].apply(clean_text)
df_test['clean_synopsis'] = df_test['synopsis'].apply(clean_text)

In [7]:
#Parameters that can be changed:
#Number of wordds in vocublary
#Class weighting


# from sklearn.feature_extraction.text import CountVectorizer
mlb=MultiLabelBinarizer()
y_train = mlb.fit_transform(df_train['genres'])
#display(y)
vectorizer = TfidfVectorizer(stop_words="english",max_features=10000,ngram_range=(1,4),binary=True)
X_train = vectorizer.fit_transform(df_train['clean_synopsis'])
#display(X_train)
#print(vectorizer.get_feature_names_out(),len(vectorizer.get_feature_names_out()))
#print(X_train.toarray())

y_val= mlb.transform(df_val['genres'])
X_val = vectorizer.transform(df_val['clean_synopsis'])
y_test= mlb.transform(df_test['genres'])
X_test = vectorizer.transform(df_test['clean_synopsis'])

from sklearn.naive_bayes import BernoulliNB

classifier = OneVsRestClassifier(BernoulliNB())#Already tried with balanced class but it is worse
classifier.fit(X_train, y_train)                                    #At least when it is not balanced it can overfit the train class

# from sklearn.metrics import classification_report
#samples	How well did we predict each movie's genres?	✅ Best for your task
#micro	    Overall genre prediction accuracy	            ✅ Good for global view
#macro	    Equal focus on rare and frequent genres	        ⚠️ Careful with imbalance
#weighted	Genre performance balanced by frequency	        ✅ Fair but skew-sensitive
#USE SAMPLES


print("Report for Train dataset")
threshold=0.2
y_pred_train=LR_OVA_prob(X_train,threshold,classifier)
print_report(y_train,y_pred_train,"samples avg","f1-score",mlb)
jaccard = jaccard_score(y_train, y_pred_train, average='samples')
print("Jaccard Similarity:", jaccard)
hr = hit_rate(y_train, y_pred_train)
print("Hit Rate:", hr)

print("----------------------")
print("Report for Val dataset")
y_pred_val=LR_OVA_prob(X_val,threshold,classifier)
print_report(y_val,y_pred_val,"samples avg","f1-score",mlb)
jaccard = jaccard_score(y_val, y_pred_val, average='samples')
print("Jaccard Similarity:", jaccard)
hr = hit_rate(y_val, y_pred_val)
print("Hit Rate:", hr)

print("----------------------")
print("Report for Test dataset")
y_pred_test=LR_OVA_prob(X_test,threshold,classifier)
print_report(y_test,y_pred_test,"samples avg","f1-score",mlb)
jaccard = jaccard_score(y_test, y_pred_test, average='samples')
print("Jaccard Similarity:", jaccard)
hr = hit_rate(y_val, y_pred_val)
print("Hit Rate:", hr)



display(result_table(y_train,y_pred_train,df_train,mlb))
display(result_table(y_val,y_pred_val,df_val,mlb))
display(result_table(y_test,y_pred_test,df_test,mlb))

Report for Train dataset
samples avg f1-score = 0.5259074834323281
Jaccard Similarity: 0.4224375409784462
Hit Rate: 0.8149880356754405
----------------------
Report for Val dataset
samples avg f1-score = 0.4249845127260669
Jaccard Similarity: 0.31515688353206733
Hit Rate: 0.7884108867427568
----------------------
Report for Test dataset
samples avg f1-score = 0.42682927709481416
Jaccard Similarity: 0.31722646636164675
Hit Rate: 0.7884108867427568


c:\Users\User\anaconda3\envs\NLP_CW\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\User\anaconda3\envs\NLP_CW\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\User\anaconda3\envs\NLP_CW\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,Synopsis,Actual Genres,Predicted Genres
0,"Sometime in the future, the world was complete...","(Action, Adventure)","(Adventure, Fantasy)"
1,"Set in 2014, the anime follows the adventures ...","(Comedy,)","(Comedy, Slice of Life)"
2,Follows a pig whose family's mission is to col...,"(Comedy, Kids)","(Comedy,)"
3,"In honor of the 2018 World Cup, this season of...","(Kids, Sci-Fi, Sports, Super Power)","(Kids, Sports)"
4,"Fairies living in a fluffy forest, where both ...","(Fantasy, Kids)","(Comedy, Kids)"
...,...,...,...
9189,The Konohagakure Grand Sports Festival has beg...,"(Action, Comedy, Fantasy, Other, Shounen, Sports)","(Action, Adventure, Comedy, Other, Shounen, Sp..."
9190,Special bundled with the Blu-ray/DVD volume of .,"(Ecchi,)","(Comedy,)"
9191,"According to the official Hobby Japan website,...","(Comedy, Ecchi, Fantasy, Parody)","(Comedy,)"
9192,A series of comedic shorts featuring chibi ver...,"(Adventure, Comedy, Fantasy, Parody)","(Comedy,)"


,Synopsis,Actual Genres,Predicted Genres
0,Shuramaru is hated and feared by the villagers...,"(Supernatural,)",()
1,The daily life of the Motsumoto family. The th...,"(Comedy, Slice of Life)","(Comedy,)"
2,"The Soul Tree, the great source of our race. ...","(Action, Adventure, Kids)","(Action, Adventure, Fantasy, Kids, Other)"
3,Himeno Mimika is a daughter of historical cook...,"(Kids, Other, Slice of Life)","(Comedy,)"
4,The third generation of J SOUL BROTHERS from E...,"(Kids, Music)","(Comedy, Kids, Music)"
...,...,...,...
1134,Specials included in the OVA of Hyakujitsu no ...,"(Other, Parody)","(Comedy,)"
1135,DVD/BD specials featuring two female cast memb...,"(Ecchi, Seinen)","(Comedy,)"
1136,"As college entrance examinations draw near, tw...","(School, Slice of Life)","(Comedy, Romance, School, Slice of Life)"
1137,Two special episodes of included on Blu-ray. 1...,"(Ecchi, Slice of Life)","(Comedy,)"


,Synopsis,Actual Genres,Predicted Genres
0,The story begins 15 years after the demon king...,"(Comedy, Fantasy, Slice of Life)","(Adventure, Fantasy)"
1,A cyborg warrior from an ancient Antarctic kin...,"(Action, Mecha, Sci-Fi)","(Action, Adventure, Sci-Fi)"
2,"The story centers on Otarou, a creature themed...","(Comedy,)","(Comedy, Slice of Life)"
3,"An elite soldier, whose nickname is God of Dea...","(Action, Adventure, Comedy, Romance, Sci-Fi)","(Action, Comedy)"
4,"Based on the real life-story of Pink Lady, a f...","(Drama, Music, Slice of Life)","(Comedy,)"
...,...,...,...
1122,An unaired episode released with the sixth BD ...,"(Sci-Fi, Seinen)","(Comedy,)"
1123,Short extras of the anime included in each of ...,"(Romance,)","(Comedy,)"
1124,Short specials included in the Blu-ray release...,"(Sci-Fi, Seinen)","(Comedy,)"
1125,Three episodes aired on TV as part of the Ribo...,"(Shoujo,)","(Comedy, School, Slice of Life)"
